In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv"
yhfinanceFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"

In [3]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [4]:
ohlc.loc[ohlc.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 11)

In [5]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [6]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [7]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [8]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [9]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [10]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
858314,2012-03-12,3MINDIA,3931.550049,4035.000000,3925.000000,4035.000000,143.0,140.0,NaN,3M India Ltd.,CONSUMER GOODS
858315,2012-03-12,AARTIDRUGS,3.962227,4.007228,3.900083,3.942941,2043.0,1461.0,NaN,Aarti Drugs Ltd.,PHARMA
858316,2012-03-12,AARTIIND,12.014886,12.180039,11.973597,11.973597,11726.0,7709.0,NaN,Aarti Industries Ltd.,CHEMICALS
858317,2012-03-12,ABB,792.433289,818.295995,764.266497,807.809877,66703.0,18444.0,340500.0,ABB India Ltd.,INDUSTRIAL MANUFACTURING
858318,2012-03-12,ABBOTINDIA,1393.245728,1419.016928,1389.080293,1405.284506,821.0,504.0,NaN,Abbott India Ltd.,PHARMA


In [11]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [12]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1903196,2012-03-12,NSENIFTY,5420.10,5421.90,5327.3,5359.55,176668237.0,0.0,31408000.0,IND_NSENIFTY,INDEX
1903197,2012-03-13,NSENIFTY,5391.05,5438.65,5390.8,5429.50,179965296.0,0.0,NaN,IND_NSENIFTY,INDEX
1903198,2012-03-14,NSENIFTY,5490.55,5499.40,5437.8,5463.90,181714070.0,0.0,32741850.0,IND_NSENIFTY,INDEX
1903199,2012-03-15,NSENIFTY,5462.50,5462.50,5362.3,5380.50,160257651.0,0.0,NaN,IND_NSENIFTY,INDEX
1903200,2012-03-16,NSENIFTY,5380.35,5445.65,5305.0,5317.90,267521147.0,0.0,34712400.0,IND_NSENIFTY,INDEX


In [13]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [14]:
finalOHLC10yrIndex.loc[finalOHLC10yrIndex.symbol == "NSE500", "date"].max()

datetime.date(2022, 3, 11)

In [15]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29435,2012-03-12,CRUDE_OIL,106.339996,107.559998,105.379997,107.500000,234727.0,NaN,NaN,OTHERS,OTHERS
29436,2012-03-12,GOLD,1699.199951,1708.099976,1694.900024,1708.099976,98.0,NaN,NaN,OTHERS,OTHERS
29437,2012-03-12,HDFC_US_ADR,15.824923,15.927259,15.745844,15.922607,1093000.0,NaN,NaN,OTHERS,OTHERS
29438,2012-03-12,ICICI_US_ADR,5.910931,5.944462,5.850257,5.912528,8518950.0,NaN,NaN,OTHERS,OTHERS
29439,2012-03-12,INFOSYS_US_ADR,5.243680,5.252786,5.171749,5.187228,13994400.0,NaN,NaN,OTHERS,OTHERS


In [16]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [17]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [31]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    # break

In [19]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/corr.csv", index=False
)

NameError: name 'corrDf' is not defined

In [18]:
allData = finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500)
allData["date"] = allData.date.apply(lambda x: x.strftime("%Y%m%d"))
allData.fillna(0, inplace=True)
allData.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\nifty500OtherOHLC10yrs.csv",
    index=False,
)

In [19]:
allData.loc[allData.symbol == "NSE500", "date"].max()

'20220311'

### Below code is for analysis stock by stock

In [70]:
symbol = "LINDEINDIA"

In [71]:
import nsepy as npy

start_date = datetime.date.today() - relativedelta(months=6)
end_date = datetime.date.today()

ohlc_data = npy.get_history(
    symbol=re.sub("&", "%26", symbol), start=start_date, end=end_date
)

In [72]:
mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"
holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = datetime.datetime.today()
from_date = today - relativedelta(months=6)
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()
oiData = pd.DataFrame()
for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )
for _, row in expriyDf.iterrows():
    logging.info(
        "fetching features data for %s for date %s exp dates %s %s %s"
        % (
            symbol,
            datetime.datetime.strftime(row["date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["next_month_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(
                row["month_after_next_month_exp_date"].date(), "%Y-%m-%d"
            ),
        )
    )
    current_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["current_exp_date"].date(),
    )
    next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["next_month_exp_date"].date(),
    )
    month_after_next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["month_after_next_month_exp_date"].date(),
    )
    if (
        (len(current_month_features["Open Interest"].values) == 1)
        and (len(next_month_features["Open Interest"].values) > 0)
        and (len(month_after_next_month_features["Open Interest"].values) > 0)
    ):
        oiData = oiData.append(
            pd.DataFrame(
                {
                    "Symbol": [symbol],
                    "Date": [row["date"].date()],
                    "cummOI": [
                        current_month_features["Open Interest"].values[0]
                        + next_month_features["Open Interest"].values[0]
                        + month_after_next_month_features["Open Interest"].values[0]
                    ],
                }
            )
        )

In [73]:
ohlc_oi_date = pd.merge(
    left=oiData,
    right=ohlc_data.reset_index(),
    left_on=["Date", "Symbol"],
    right_on=["Date", "Symbol"],
    how="inner",
)

KeyError: 'Date'

In [ ]:
ohlc_oi_date["vol_per_trade"] = ohlc_oi_date["Volume"] / ohlc_oi_date["Trades"]
ohlc_oi_date.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_oi_date.csv",
    index=False,
)

### download FII data

In [38]:
endDate = datetime.date.today()
startDate = endDate - relativedelta(months=6)
dtSrs = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%b%d%Y")
)
dtSrsLong = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%B%d%Y")
)
dateDf = pd.DataFrame({"shrtDt": dtSrs, "longDt": dtSrsLong})
data = pd.DataFrame()

for _, dd in dateDf.iterrows():
    urlShrt = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["shrtDt"])
    )
    urlLng = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["longDt"])
    )
    if data.shape[0] == 0:
        try:
            data = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["shrtDt"]]

        except:
            data = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["longDt"]]

    else:
        try:
            temp = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["shrtDt"]]
        except:
            temp = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["longDt"]]
        data = pd.merge(
            left=data, right=temp, left_on="Sector", right_on="Sector", how="outer"
        )

data.to_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "fii.csv", index=False)